### Importing all the required library for webscarpping at once

In [6]:
!pip install selenium 
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')
import time
from selenium.common.exceptions import NoSuchElementException

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars

In [17]:
#Calling the automated chrome driver
driver = webdriver.Chrome(r'C:\Users\Vinit\Downloads\chromedriver.exe')


#URL to be web scrape

url = 'https://www.amazon.in/'
driver.get(url)

#Input from user
product = input('Enter product to search on amazon.in :  ')
#Search sunglasses product
search_product = driver.find_element(By.XPATH,"//input[@id='twotabsearchtextbox']")
search_product.send_keys(product)


#clicking the search button
search_button = driver.find_element(By.XPATH,"//input[@id='nav-search-submit-button']")
search_button.click()
print("search completed in chrome browser !!")

Enter product to search on amazon.in :  tv
search completed in chrome browser !!


## Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [21]:
pageTags = driver.find_elements(By.XPATH,"//a[contains (@aria-label,'Go to page')]")
                                
BrandName = []
NameProduct = []
Price = []
ExpectedDelivery = []
Availability = []
ProductURL = []

for p in range(3):   
    productTags = driver.find_elements(By.XPATH,"//h2[contains(@class,'a-size-mini a-spacing-none a-color-base s-line-clamp')]/a")

    for i in range(len(productTags)):
    #for i in range(2):    
        new_url = productTags[i].get_attribute('href')


        # Open a new window
        driver.execute_script("window.open('');")

        # Switch to the new window and open new URL
        driver.switch_to.window(driver.window_handles[1])
        driver.get(new_url)
        
        try:
            brandTag = driver.find_element(By.XPATH,"//tr[@class='a-spacing-small po-brand']/td/span[@class='a-size-base']")
            BrandName.append(brandTag.text)
        except NoSuchElementException:
            BrandName.append('Not available')
            
        try:
            PriceTag = driver.find_element(By.XPATH,"//span[@class='a-price aok-align-center reinventPricePriceToPayMargin priceToPay']/span/span[@class='a-price-whole']")
            Price.append(PriceTag.text)
        except NoSuchElementException:
            Price.append('Not available') 

        try:
            nameTag = driver.find_element(By.XPATH,"//span[@id='productTitle']")
            NameProduct.append(nameTag.text)
        except NoSuchElementException:
            NameProduct.append('Not available') 

        try:
            deliveryTag = driver.find_element(By.XPATH,"//div[@class='a-spacing-base']/span/span[@class='a-text-bold']")
            ExpectedDelivery.append( deliveryTag.text)
        except NoSuchElementException:
            ExpectedDelivery.append('Not available') 

        try:
            AvailibilityTag = driver.find_element(By.XPATH,"//div[@id='availability']/span[@class='a-size-medium a-color-success']")
            Availability.append(AvailibilityTag.text)
        except NoSuchElementException:
            Availability.append('Not available') 

        urlTag = driver.current_url
        ProductURL.append(urlTag)


        # Closing new_url tab
        driver.close()

        # Switching to old tab
        driver.switch_to.window(driver.window_handles[0])
        #print('total products scrapped from page',p+1,'is', i)
    try:    
        nextButton = driver.find_element(By.XPATH,"//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
        nextButton.click()
        time.sleep(10)
    except NoSuchElementException:
        print ('Search completed for the pages available !!!')

print('Search completed for the first 3 pages !!!')
driver.close()
    


Search completed for the first 3 pages !!!


### Dataframe for amazon search results !

In [22]:
dict = {'BrandName':BrandName,'Product Name':NameProduct,'Product Price':Price,'Delivery':ExpectedDelivery,
       'Availability':Availability,'ProductURL':ProductURL}

Amazondf = pd.DataFrame(dict)

Amazondf.tail(10)

,BrandName,Product Name,Product Price,Delivery,Availability,ProductURL
0,Hisense,Hisense 108 cm (43 inches) Bezelless Series 4K...,"27,990","Saturday, 3 December, 7 AM - 9 PM",In stock.,https://www.amazon.in/Hisense-inches-Bezelless...
1,Intex,Intex 108 cm (43 inches) 4K Ultra HD Smart LED...,"26,998","Saturday, 3 December, 7 AM - 9 PM",In stock.,https://www.amazon.in/Intex-inches-Ultra-Smart...
2,SANSUI,Sansui 80cm (32 inches) HD Ready Certified And...,"11,990","Saturday, 3 December, 7 AM - 9 PM",In stock.,https://www.amazon.in/Sansui-Certified-Android...
3,Intex,Intex 164 cm (65 inches) 4K Ultra HD Smart LED...,"63,999","Saturday, 3 December, 7 AM - 9 PM",In stock.,https://www.amazon.in/Intex-inches-Ultra-Smart...
4,Hisense,Hisense 139 cm (55 inches) Tornado 2.0 Series ...,"42,990","Saturday, 3 December, 7 AM - 9 PM",In stock.,https://www.amazon.in/Hisense-inches-Tornado-G...
5,TCL,TCL 138.7 cm (55 inches) Metallic Bezel-Less S...,"38,990","Saturday, 3 December, 7 AM - 9 PM",In stock.,https://www.amazon.in/TCL-inches-Metallic-Beze...


### Q.4Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
#Calling the automated chrome driver
driver = webdriver.Chrome(r'C:\Users\Vinit\Downloads\chromedriver.exe')


#URL to be web scrape

url = 'https://www.flipkart.com/'
driver.get(url)

try:
    cross_button = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/button')
    cross_button.click()
    #Input from user
except NoSuchElementException:
    pass
    
product = input('Enter smartphone name to search on flipkart :  ')
#Search sunglasses product
search_product = driver.find_element(By.XPATH,"//input[@class='_3704LK']")
search_product.send_keys(product)


#clicking the search button
search_button = driver.find_element(By.XPATH,"//button[@class='L0Z3Pu']")
search_button.click()
print("showing search results in chrome browser !!")


BrandName = []
SmartphoneName = []
Colour = []
RAM = []
Storage = []
PrimaryCamera = []
SecondaryCamera = []
DisplaySize = []
BatteryCapacity = []
Price = []
ProductURL = []


In [ ]:
productTags = driver.find_elements(By.XPATH,"//div[contains(@data-id,'MOB')]/div/a[1]")

for i in range(len(productTags)):
#for i in range(2):    
    new_url = productTags[i].get_attribute('href')


    # Open a new window
    driver.execute_script("window.open('');")

    # Switch to the new window and open new URL
    driver.switch_to.window(driver.window_handles[1])
    driver.get(new_url)
    time.sleep(5)
    try:
        brandTag = driver.find_element(By.XPATH,"(//a[@class='_2whKao'])[4]")
        BrandName.append(brandTag.text)
    except NoSuchElementException:
        BrandName.append('-')
    try:
        SmartphoneNameTag = driver.find_element(By.XPATH,"(//li[@class='_21lJbe'])[3]")
        SmartphoneName.append(SmartphoneNameTag.text)
    except NoSuchElementException:
        SmartphoneName.append('-') 
    try:
        ColourTag = driver.find_element(By.XPATH,"(//li[@class='_21lJbe'])[4]")
        Colour.append(ColourTag.text)
    except NoSuchElementException:
        Colour.append('-') 
    try:
        RAMTag = driver.find_element(By.XPATH,"//*[@id='container']/div/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/ul/li[1]")
        MemoryList =RAMTag.text.split("|")
        RAM.append(MemoryList[0])
        Storage.append(MemoryList[1])
    except NoSuchElementException:
        RAM.append('Not available') 
        torage.append('Not available')
    try:
        CameraTag = driver.find_element(By.XPATH,"//*[@id='container']/div/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/ul/li[3]")
        cameraList = CameraTag.text.split("|")
        if len(cameraList)==2:
            PrimaryCamera.append(cameraList[0])
            SecondaryCamera.append(cameraList[1])
        else:
            PrimaryCamera.append(cameraList[0])
            SecondaryCamera.append('-')
    except NoSuchElementException:
        PrimaryCamera.append('-')
        SecondaryCamera.append('-')
    try:
        DisplaySizeTag = driver.find_element(By.XPATH,"//*[@id='container']/div/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/ul/li[2]")
        DisplaySize.append( DisplaySizeTag.text)
    except NoSuchElementException:
        DisplaySize.append('-')
    
    try:
        BatteryCapacityTag = driver.find_element(By.XPATH,"//*[@id='container']/div/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/ul/li[4]")
        BatteryCapacity.append(BatteryCapacityTag.text)
    except NoSuchElementException:
        BatteryCapacity.append('-') 
        
    try:
        PriceTag = driver.find_element(By.XPATH,"(//div[@class='_30jeq3'])[1]")
        Price.append(PriceTag.text)
    except NoSuchElementException:
        Price.append('Not available') 
    
    urlTag = driver.current_url
    ProductURL.append(urlTag)
    
    
    # Closing new_url tab
    driver.close()

    # Switching to old tab
    driver.switch_to.window(driver.window_handles[0])
    #print('total products scrapped from page',p+1,'is', i)
driver.close()

In [ ]:
dict = {'BrandName':BrandName,'Product Name':SmartphoneName,
        'Colour':Colour,'RAM':RAM,'Storage':Storage,'PrimaryCamera':PrimaryCamera,'SecondaryCamera':SecondaryCamera,
        'DisplaySize':DisplaySize,'BatteryCapacity':BatteryCapacity,
        'Product Price':Price,'ProductURL':ProductURL}

Flipkartdf = pd.DataFrame(dict)
Flipkartdf.to_csv('fipkart.csv')
Flipkartdf.tail(10)
